In [1]:
import json
from pathlib import Path
from typing import (
    Optional, Sequence, List, Tuple, Dict, Union, Callable
)

In [2]:
FILE_DIR = Path("/home/marmot/Boyang/JARVIS-1/jarvis/assets")

SPAWN_FILE = FILE_DIR / "spawn.json"
MC_CONSTANTS_FILE = FILE_DIR / "mc_constants.1.16.json"
COMMON_LABELS_FILE = FILE_DIR / "common_labels.json"
CARED_ITEMS_FILE = FILE_DIR / "cared_items.json"
RECIPES_DIR = FILE_DIR / "recipes"
COLOR_FILE = FILE_DIR / "colors.json"
TAG_ITEMS_FILE = FILE_DIR / "tag_items.json"
SKILL_FILE = FILE_DIR / "skill.json"
MEMORY_FILE = FILE_DIR / "memory.json"
TASKS_FILE = FILE_DIR / "tasks.json"

RECIPES_BOOKS = {}
RECIPES_INGREDIENTS = []

for recipe in Path(RECIPES_DIR).glob("*.json"):
    with recipe.open("r") as f:
        RECIPES_BOOKS[recipe.stem] = json.load(f)
        if RECIPES_BOOKS[recipe.stem]["type"] == "minecraft:crafting_shapeless":
            for ingredient in RECIPES_BOOKS[recipe.stem]["ingredients"]:
                if 'item' in ingredient:
                    RECIPES_INGREDIENTS.append(ingredient['item'].replace("minecraft:", ""))
                elif 'tag' in ingredient:
                    RECIPES_INGREDIENTS.append(ingredient['tag'].replace("minecraft:", ""))
        elif RECIPES_BOOKS[recipe.stem]["type"] == "minecraft:crafting_shaped":
            for key, ingredient in RECIPES_BOOKS[recipe.stem]["key"].items():
                if 'item' in ingredient:
                    RECIPES_INGREDIENTS.append(ingredient['item'].replace("minecraft:", ""))
                elif 'tag' in ingredient:
                    RECIPES_INGREDIENTS.append(ingredient['tag'].replace("minecraft:", ""))

RECIPES_INGREDIENTS = sorted(list(set(RECIPES_INGREDIENTS)))
# print('Len of ingredients:', len(RECIPES_INGREDIENTS))
# print(RECIPES_INGREDIENTS)

with open(SPAWN_FILE, "r") as f:
    SPAWN_CONSTANTS = json.load(f)
    
with open(COLOR_FILE, "r") as f:
    COLOR_CONSTANTS = json.load(f)

def get_spawn_position(
    seed: Optional[int] = None, 
    biome: Optional[str] = None, 
    **kwargs,
) -> List[Tuple[int, int, int]]:
    result = []
    for spaw_position in SPAWN_CONSTANTS:
        if seed:
            if spaw_position['seed'] != seed:
                continue
        if biome:
            if spaw_position['biome'] != biome:
                continue
        result.append(spaw_position['player_pos'])
    return result

ALL_SEEDS = sorted(list(set([spaw_position['seed'] for spaw_position in SPAWN_CONSTANTS])))

with open(MC_CONSTANTS_FILE, "r") as f:
    MC_CONSTANTS = json.load(f)

with open(COMMON_LABELS_FILE, "r") as f:
    COMMON_ITEMS_IDX_TO_NAME = json.load(f)

with open(CARED_ITEMS_FILE, "r") as f:
    CARED_ITEMS = json.load(f)

ALL_BIOMES = ['forest', 'plains']
ALL_WEATHERS = ['clear', 'rain', 'thunder']
EQUIP_SLOTS = ['head', 'feet', 'chest', 'legs', 'offhand']

ALL_ITEMS = {}
EQUIPABLE_ITEMS = {}
for item in MC_CONSTANTS["items"]:
    ALL_ITEMS[item['type']] = item
    if item['bestEquipmentSlot'] in EQUIP_SLOTS:
        equip_slot = item['bestEquipmentSlot']
        if equip_slot not in EQUIPABLE_ITEMS:
            EQUIPABLE_ITEMS[equip_slot] = []
        EQUIPABLE_ITEMS[equip_slot].append(item['type'])
    
ALL_ITEMS_IDX_TO_NAME = sorted(list(ALL_ITEMS.keys()))
ALL_ITEMS_NAME_TO_IDX = {item: idx for idx, item in enumerate(ALL_ITEMS_IDX_TO_NAME)}


KEYS_TO_INFO = [
    'pov', 
    'inventory', 
    'equipped_items', 
    'life_stats', 
    'location_stats', 
    'use_item', 
    'drop', 
    'pickup', 
    'break_item', 
    'craft_item', 
    'mine_block', 
    'damage_dealt', 
    'entity_killed_by', 
    'kill_entity', 
    'full_stats', 
    'player_pos', 
    'is_gui_open'
]

In [4]:
print('Len of recipes:', len(RECIPES_BOOKS))
print('Len of ingredients:', len(RECIPES_INGREDIENTS))

# 860 recipies, with need of 265 unique items

Len of recipes: 860
Len of ingredients: 265


In [3]:
with open('full_recipies.json', 'w') as json_file:
    json.dump(RECIPES_BOOKS, json_file, indent=4)

with open('full_ingredients.json', 'w') as json_file:
    json.dump(RECIPES_INGREDIENTS, json_file, indent=4)

In [5]:
ALL_ITEMS_IDX_TO_NAME 
ALL_ITEMS_NAME_TO_IDX 
print('Len of all items:', len(ALL_ITEMS))
print('Len of equipable items:', len(EQUIPABLE_ITEMS))

Len of all items: 976
Len of equipable items: 5


In [6]:
print("item property")
list(ALL_ITEMS['air'].keys())

item property


['type',
 'damageable',
 'stackable',
 'stackSize',
 'useAction',
 'enchantable',
 'rarity',
 'maxDamage',
 'maxUseDuration',
 'block',
 'hasContainerItem',
 'bestEquipmentSlot']

In [7]:
EQUIPABLE_ITEMS.keys()

dict_keys(['head', 'chest', 'legs', 'feet', 'offhand'])

In [8]:
print("Head equipments: ",len(EQUIPABLE_ITEMS['head']))
print("Chest equipments: ",len(EQUIPABLE_ITEMS['chest']))
print("Legs equipments: ",len(EQUIPABLE_ITEMS['legs']))
print("Feet equipments: ",len(EQUIPABLE_ITEMS['feet']))
print("offhand equipments: ",len(EQUIPABLE_ITEMS['offhand']))

Head equipments:  14
Chest equipments:  7
Legs equipments:  6
Feet equipments:  6
offhand equipments:  1


In [9]:
len(CARED_ITEMS)

78

In [10]:
print(set(RECIPES_BOOKS.keys()).issubset(set(ALL_ITEMS.keys())))
len(set(RECIPES_BOOKS.keys()) & set(ALL_ITEMS.keys()))

False


605

In [11]:
print(set(RECIPES_INGREDIENTS).issubset(set(ALL_ITEMS.keys())))
len(set(RECIPES_INGREDIENTS) & set(ALL_ITEMS.keys()))

False


249

In [12]:
print(set(CARED_ITEMS).issubset(set(ALL_ITEMS.keys())))
print(set(CARED_ITEMS).issubset(set(RECIPES_INGREDIENTS)))
print(set(CARED_ITEMS).issubset(set(RECIPES_BOOKS.keys())))

True
False
False


In [13]:
filtered_dict = {key: ALL_ITEMS[key] for key in CARED_ITEMS if key in ALL_ITEMS}

# Save the filtered dictionary as a JSON file
with open('cared_items_with_property.json', 'w') as json_file:
    json.dump(filtered_dict, json_file, indent=4)

In [14]:
filtered_ingredients = list(set(CARED_ITEMS).intersection(RECIPES_INGREDIENTS))

In [15]:
# Save the filtered dictionary as a JSON file
with open('cared_ingredients.json', 'w') as json_file:
    json.dump(filtered_ingredients, json_file, indent=4)

In [16]:
filtered_recipes_keys = list(set(CARED_ITEMS).intersection(set(RECIPES_BOOKS.keys())))
filtered_recipes = {key: RECIPES_BOOKS[key] for key in filtered_recipes_keys}

In [17]:
keys = set()
for aim, recipy in filtered_recipes.items():
    for k in recipy.keys():
        keys.add(k)

In [18]:
keys

{'cookingtime',
 'experience',
 'group',
 'ingredient',
 'ingredients',
 'key',
 'pattern',
 'result',
 'type'}

In [19]:
keys_to_remove = []
for aim, recipy in filtered_recipes.items():

    if 'key' in  recipy.keys():
        for k, v in recipy['key'].items():
            try:
                t = v['item'].replace("minecraft:", "")
            except:
                t = v['tag'].replace("minecraft:", "")
            if t not in CARED_ITEMS:
                print(aim, t)
                keys_to_remove.append(aim)
    
    if 'ingredient' in  recipy.keys():
        for k, v in recipy['ingredient'].items():
            if k in ['item', 'tag']:   
                t = v.replace("minecraft:", "")
            if t not in CARED_ITEMS:
                print(aim, t)
                keys_to_remove.append(aim)

    if 'ingredients' in  recipy.keys():
        for v in recipy['ingredients']:
            try:
                t = v['item'].replace("minecraft:", "")
            except:
                t = v['tag'].replace("minecraft:", "")
            if t not in CARED_ITEMS:
                print(aim, t)
                keys_to_remove.append(aim)
    
    if 'result' in  recipy.keys():
        if isinstance(recipy['result'], str):
            t = recipy['result'].replace("minecraft:", "")
        else:
            for k, v in recipy['result'].items():
                if k in ['item', 'tag']:   
                    t = v.replace("minecraft:", "")

        if t not in CARED_ITEMS:
            print(aim, t)
            keys_to_remove.append(aim)

torch coals
stone_shovel stone_tool_materials
crafting_table planks
wooden_shovel planks
dark_oak_planks dark_oak_logs
stone_sword stone_tool_materials
oak_planks oak_logs
yellow_bed yellow_wool
yellow_bed planks
wooden_sword planks
spruce_planks spruce_logs
jungle_planks jungle_logs
acacia_planks acacia_logs
wooden_pickaxe planks
gold_ingot gold_ores
stick planks
leather rabbit_hide
birch_planks birch_logs
stone_axe stone_tool_materials
diamond diamond_block
white_bed planks
orange_bed orange_dye
furnace stone_crafting_materials
red_bed red_wool
red_bed planks
coal coal_block
wooden_axe planks
stone_pickaxe stone_tool_materials


In [20]:
len(set(keys_to_remove)), len(filtered_recipes)

(26, 56)

In [21]:
for k in set(keys_to_remove):
    del filtered_recipes[k]

# Save the filtered dictionary as a JSON file
with open('cared_recipies.json', 'w') as json_file:
    json.dump(filtered_recipes, json_file, indent=4)

In [22]:
with open(TASKS_FILE, 'r') as f:
    jarvis_tasks = json.load(f)

with open(SKILL_FILE, 'r') as f:
    jarvis_skills = json.load(f)


In [25]:
skill_type = set()
for skill_object in jarvis_skills:
    for act_choice in jarvis_skills[skill_object]:
        skill_type.add(act_choice['type'])

In [26]:
skill_type

{'equip', 'mine'}

In [ ]:
'''
explore
chop down
equip
break
mine
collect
get
dig
gather
dig down
obtain
kill
combat
'''